## Mulitple Different Time Series Anomaly Detection Methods on the WaDi Data Set

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from statsmodels.graphics.tsaplots import plot_acf
import math
from sklearn.preprocessing import MinMaxScaler
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
import random
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
from tabulate import tabulate

2026-01-08 16:06:09.626359: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
attack_df = pd.read_csv("/home/brandon/data/WADI_attackdataLABLE.csv", header = 1)
attack_df['Date'] = attack_df['Date ']
attack_df = attack_df[attack_df['Row '] <= 172801.0]
attack_df['Attack_Label'] = attack_df['Attack LABLE (1:No Attack, -1:Attack)']
attack_df = attack_df.drop(columns = ['Row ', 'Date ', 'Attack LABLE (1:No Attack, -1:Attack)'])
# attack_df['Date'] = pd.to_datetime(attack_df['Date'], format='%m/%d/%y')
attack_df

,Time,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LS_001_AL,1_LS_002_AL,1_LT_001_PV,...,3_MV_003_STATUS,3_P_001_STATUS,3_P_002_STATUS,3_P_003_STATUS,3_P_004_STATUS,LEAK_DIFF_PRESSURE,PLANT_START_STOP_LOG,TOTAL_CONS_REQUIRED_FLOW,Date,Attack_Label
0,00:00.0,164.210,0.529486,11.9972,482.480,0.331167,0.001273,0.0,0.0,48.4820,...,1.0,1.0,1.0,1.0,1.0,62.6226,1.0,0.39,10/9/17,1
1,00:01.0,164.210,0.529486,11.9972,482.480,0.331167,0.001273,0.0,0.0,48.4820,...,1.0,1.0,1.0,1.0,1.0,62.6226,1.0,0.39,10/9/17,1
2,00:02.0,164.210,0.529486,11.9972,482.480,0.331167,0.001273,0.0,0.0,48.4820,...,1.0,1.0,1.0,1.0,1.0,62.6226,1.0,0.39,10/9/17,1
3,00:03.0,164.210,0.529486,11.9972,482.480,0.331167,0.001273,0.0,0.0,48.4820,...,1.0,1.0,1.0,1.0,1.0,62.6226,1.0,0.39,10/9/17,1
4,00:04.0,164.210,0.529486,11.9972,482.480,0.331167,0.001273,0.0,0.0,48.4820,...,1.0,1.0,1.0,1.0,1.0,62.6226,1.0,0.39,10/9/17,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172796,59:56.0,172.959,0.547483,11.9184,466.034,0.318217,0.001222,0.0,0.0,55.5587,...,1.0,1.0,1.0,1.0,1.0,59.3546,1.0,0.00,10/11/17,1
172797,59:57.0,172.959,0.547483,11.9184,466.034,0.318217,0.001222,0.0,0.0,55.5587,...,1.0,1.0,1.0,1.0,1.0,59.3546,1.0,0.00,10/11/17,1
172798,59:58.0,172.915,0.583479,11.9211,466.051,0.318317,0.001260,0.0,0.0,55.7260,...,1.0,1.0,1.0,1.0,1.0,58.8102,1.0,0.00,10/11/17,1
172799,59:59.0,172.915,0.583479,11.9211,466.051,0.318317,0.001260,0.0,0.0,55.7260,...,1.0,1.0,1.0,1.0,1.0,58.8102,1.0,0.00,10/11/17,1


In [3]:
Train_df = pd.read_csv("/home/brandon/data/WADI_14days_new.csv")
Train_df = Train_df.drop(columns = ['Row'])
Train_df['Date'] = pd.to_datetime(Train_df['Date'], format='mixed')
Train_df['Attack_Label'] = np.zeros(Train_df.shape[0], dtype='float32')
Train_df

,Date,Time,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LS_001_AL,1_LS_002_AL,...,3_MV_002_STATUS,3_MV_003_STATUS,3_P_001_STATUS,3_P_002_STATUS,3_P_003_STATUS,3_P_004_STATUS,LEAK_DIFF_PRESSURE,PLANT_START_STOP_LOG,TOTAL_CONS_REQUIRED_FLOW,Attack_Label
0,2017-09-25,00:00.0,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,0,...,1,1,1,1,1,1,67.9651,1,0.68,0.0
1,2017-09-25,00:01.0,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,0,...,1,1,1,1,1,1,67.9651,1,0.68,0.0
2,2017-09-25,00:02.0,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,0,...,1,1,1,1,1,1,67.9651,1,0.68,0.0
3,2017-09-25,00:03.0,171.155,0.607477,11.5725,504.673,0.318438,0.001207,0,0,...,1,1,1,1,1,1,67.1948,1,0.68,0.0
4,2017-09-25,00:04.0,171.155,0.607477,11.5725,504.673,0.318438,0.001207,0,0,...,1,1,1,1,1,1,67.1948,1,0.68,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784566,2017-10-07,16:06.0,175.855,0.589478,11.8941,479.191,0.331571,0.001128,0,0,...,1,1,1,1,1,1,60.6305,1,0.25,0.0
784567,2017-10-07,16:07.0,175.855,0.589478,11.8941,479.191,0.331571,0.001128,0,0,...,1,1,1,1,1,1,60.6305,1,0.25,0.0
784568,2017-10-07,16:08.0,175.855,0.589478,11.8941,479.191,0.331571,0.001128,0,0,...,1,1,1,1,1,1,60.6305,1,0.25,0.0
784569,2017-10-07,16:09.0,175.896,0.613476,11.8913,479.224,0.331622,0.001173,0,0,...,1,1,1,1,1,1,60.4477,1,0.25,0.0


In [5]:
Val_df = attack_df.loc[attack_df['Date'] == '10/9/17'].copy()

# 3. Create the Test DataFrame (Test_df) as the remainder
Test_df = attack_df.loc[attack_df['Date'] != '10/9/17'].copy()


In [6]:
# Drop columns that are always NaN
# Identify columns that are always NaN in the training set
always_nan_cols = Train_df.columns[Train_df.isnull().all()].tolist()

print(f"\nNumber of columns that are always NaN in training data: {len(always_nan_cols)}")
print("Always-NaN columns:")
print(always_nan_cols)

Train_df = Train_df.drop(columns=always_nan_cols)
Val_df   = Val_df.drop(columns=always_nan_cols)
Test_df  = Test_df.drop(columns=always_nan_cols)

print("\nAfter dropping always-NaN columns:")
print(f"Train shape: {Train_df.shape}")
print(f"Val shape:   {Val_df.shape}")
print(f"Test shape:  {Test_df.shape}")


Number of columns that are always NaN in training data: 4
Always-NaN columns:
['2_LS_001_AL', '2_LS_002_AL', '2_P_001_STATUS', '2_P_002_STATUS']

After dropping always-NaN columns:
Train shape: (784571, 126)
Val shape:   (21600, 126)
Test shape:  (151201, 126)


In [7]:
# Use appropriate methods to handle missing values
# Use liner interpolation for missing values that spread out across the data
Train_df['1_AIT_002_PV'] = Train_df['1_AIT_002_PV'].interpolate(method='linear')
Train_df['1_AIT_004_PV'] = Train_df['1_AIT_004_PV'].interpolate(method='linear')
Train_df['2B_AIT_004_PV'] = Train_df['2B_AIT_004_PV'].interpolate(method='linear')
# Use forward fill for missing values that occur in blocks
Train_df['3_AIT_004_PV'] = Train_df['3_AIT_004_PV'].ffill()

In [18]:
# 4a. Find features using Train_df (as before)
features_original = [c for c in Train_df.columns if c not in ['Date', 'Time']]
constant_features = [col for col in features_original if Train_df[col].nunique() == 1]
features_final = [col for col in features_original if col not in constant_features]
print(f"Number of constant features removed: {len(constant_features)}")

scaler = MinMaxScaler()

# Transform the data using the NumPy arrays X_train, X_val, X_test
Train_scaled = scaler.fit_transform(Train_df[features_final])
Val_scaled = scaler.transform(Val_df[features_final])
Test_scaled = scaler.transform(Test_df[features_final])

y_train = Train_df['Attack_Label'].values.astype('float32')
y_val = Val_df['Attack_Label'].values.astype('float32')
y_test = Test_df['Attack_Label'].values.astype('float32')

Number of constant features removed: 31


In [21]:
def make_window_dataset(X, window_size, batch_size, stride=1):
    ds = tf.data.Dataset.from_tensor_slices(X)

    ds = (
        ds
        .window(window_size, shift=stride, drop_remainder=True)
        .flat_map(lambda w: w.batch(window_size))
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    return ds

In [22]:
X_train = Train_scaled.astype("float32")
X_val   = Val_scaled.astype("float32")
X_test  = Test_scaled.astype("float32")

In [26]:
WINDOW_SIZE = 30
BATCH_SIZE = 32
STRIDE = 1  # increase later if needed

train_ds = make_window_dataset(
    X_train,
    y_train,
    WINDOW_SIZE,
    BATCH_SIZE
)

# val_ds = make_window_dataset(
#     X_val,
#     y_val,
#     WINDOW_SIZE,
#     BATCH_SIZE,
#     stride=STRIDE
#     )

I0000 00:00:1767918762.387629  566901 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22438 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6
I0000 00:00:1767918762.388081  566901 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22453 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:21:00.0, compute capability: 8.6
2026-01-08 17:32:42.999142: W tensorflow/core/framework/op_kernel.cc:1855] OP_REQUIRES failed at window_dataset_op.cc:417 : INVALID_ARGUMENT: size must be a scalar
2026-01-08 17:32:42.999166: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: size must be a scalar


InvalidArgumentError: {{function_node __wrapped__WindowDataset_device_/job:localhost/replica:0/task:0/device:CPU:0}} size must be a scalar [Op:WindowDataset] name: 

In [ ]:
# # --- Create Label Windows ---
# # This will also create (N - n_window + 1) labels
# y_train_window_labels = convert_labels_to_windows(y_train_tensor, n_window)
# y_val_window_labels = convert_labels_to_windows(y_val_tensor, n_window)
# y_test_window_labels = convert_labels_to_windows(y_test_tensor, n_window)

In [61]:
#GRU
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = '42'
tf.config.experimental.enable_op_determinism()

GRU_model = keras.Sequential([
    layers.GRU(units = 32, return_sequences=False,
    input_shape=(train_windows.shape[1], train_windows.shape[2])),
    layers.RepeatVector(train_windows.shape[1]),

    layers.GRU(units = 32, return_sequences=True),
    layers.TimeDistributed(layers.Dense(train_windows.shape[2]))

])

optimizer = keras.optimizers.Adam(learning_rate=1e-3)
GRU_model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

GRU_model.summary()

I0000 00:00:1767892289.655252  564530 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22438 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6
I0000 00:00:1767892289.655737  564530 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22453 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:21:00.0, compute capability: 8.6
/home/brandon/miniconda3/envs/ml/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 32)             │        12,576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 30, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 30, 32)         │         6,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 30, 97)         │         3,201 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,113 (86.38 KB)

 Trainable params: 22,113 (86.38 KB)

 Non-trainable params: 0 (0.00 B)

In [62]:
# Train the GRU
X_train = train_windows
X_val = validation_windows

GRU_history = GRU_model.fit(
    X_train,X_train,
    validation_data = (X_val, X_val),
    epochs = 30,
    batch_size = 64,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor='val_loss', patience = 10, restore_best_weights = True)
    ],
    shuffle = False
)

2026-01-08 10:11:59.663997: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 9132068880 exceeds 10% of free system memory.
2026-01-08 10:12:03.567918: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 9132068880 exceeds 10% of free system memory.


Epoch 1/30


2026-01-08 10:12:07.574314: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 9132068880 exceeds 10% of free system memory.
2026-01-08 10:12:12.164133: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 9132068880 exceeds 10% of free system memory.
2026-01-08 10:12:17.990479: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapData

12252/12259 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: nan - mae: nan

2026-01-08 10:13:54.135885: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 251086440 exceeds 10% of free system memory.
2026-01-08 10:13:54.668001: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


12259/12259 ━━━━━━━━━━━━━━━━━━━━ 109s 8ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 2/30
 5583/12259 ━━━━━━━━━━━━━━━━━━━━ 51s 8ms/step - loss: nan - mae: nan

KeyboardInterrupt: 

In [ ]:
# Run GRU predictions
GRU_X_val_pred = GRU_model.predict(X_val)

# X_val is your validation_windows tensor
X_val_np = X_val.numpy()

GRU_y_val_np = y_val_window_labels.numpy()

# Compute reconstruction error per sequence
GRU_reconstruction_error = np.mean(np.square(GRU_X_val_pred - X_val_np), axis=(1, 2))

GRU_BEST_THRESHOLD = 0.20

GRU_y_pred_labels = (GRU_reconstruction_error > GRU_BEST_THRESHOLD).astype(int)
GRU_prec, GRU_rec, GRU_f1, _ = precision_recall_fscore_support(
    GRU_y_val_np , GRU_y_pred_labels, average='binary', zero_division=0
)

# Calculate mean and std for reconstruction errors for tuning.
GRU_mean_error = np.mean(GRU_reconstruction_error)
GRU_std_error = np.std(GRU_reconstruction_error)
GRU_training_error = GRU_history.history['loss'][-1]
GRU_validation_error = GRU_history.history['val_loss'][-1]

print(f"GRU Metrics: Mean Error = {GRU_mean_error:.4f}, Std Dev = {GRU_std_error:.4f}, Training Error = {GRU_training_error:.4f}, Validation Error = {GRU_validation_error:.4f}")
print(f"Precision: {GRU_prec:.3f} | Recall: {GRU_rec:.3f} | F1: {GRU_f1:.3f}")

In [ ]:
loss_key = 'loss'
val_loss_key = 'val_loss'

plt.figure(figsize=(8,5))
plt.plot(GRU_history.history[loss_key], label='Training Loss')
plt.plot(GRU_history.history[val_loss_key], label='Validation Loss')
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.title("Training vs Validation Loss (Transformer Autoencoder)")
plt.legend()
plt.grid(True)
plt.show()